In [1]:
import pandas as pd

## 데이터셋 불러오기

In [2]:
keywords = pd.read_csv('./dataset/금지어keyword.csv')
blacklist = pd.read_csv('./dataset/블랙리스트정보v2 - 복사본.csv', encoding='latin1')
normal_item_2022 = pd.read_csv('./dataset/정상상품.csv')
weird_item_2022 = pd.read_csv('./dataset/이상상품.csv')
normal_member = pd.read_csv('./dataset/정상회원데이터.csv')
weird_member = pd.read_csv('./dataset/이상회원데이터.csv')
inquir = pd.read_csv('./dataset/인콰이어리데이터.csv')

FileNotFoundError: [Errno 2] No such file or directory: './dataset/블랙리스트정보v2 - 복사본.csv'

In [ ]:
weird_member['MEMBER_ID'].value_counts()

## 이상회원 -> 이상상품, 정상회원 -> 정상상품

In [ ]:
weird_item_2022.shape

In [ ]:
weird_df = weird_item_2022[weird_item_2022['MEMBER_ID'].isin(weird_member['MEMBER_ID'])]

# common_rows now contains only the rows with common 'MEMBER_ID' values
weird_df.shape

In [ ]:
normal_df = normal_item_2022[normal_item_2022['MEMBER_ID'].isin(normal_member['MEMBER_ID'])]

## Feature Eng

In [ ]:
weird_df['DISPLAY'] = weird_df['DISPLAY'].replace({'Y': 0, 'N': 1})

In [ ]:
normal_df['DISPLAY'] = normal_df['DISPLAY'].replace({'Y': 0, 'N': 1})

In [ ]:
weird_df['DISPLAY'].value_counts()

In [ ]:
normal_df['DISPLAY'].value_counts()

In [ ]:
weird_df['SHOW_CHK'] = weird_df['SHOW_CHK'].replace({'S': 0, 'P': 1, 'R': 2, 'B': 3})

In [ ]:
weird_df['SHOW_CHK'] = weird_df['SHOW_CHK'].replace({'S': 0, 'P': 1, 'R': 2, 'B': 3})

In [ ]:
normal_df['SHOW_CHK'] = normal_df['SHOW_CHK'].replace({'A': 0})

In [ ]:
normal_df['SHOW_CHK'].value_counts()

In [ ]:
weird_df.head()

## 종속변수 컬럼 추가

In [ ]:
weird_df['judge'] = 0
normal_df['judge'] = 1

## 패키지 로드

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

## 데이터 클렌징

### 데이터 드롭

In [ ]:
normal_df = normal_df.drop('MEMBER_ID', axis=1)
weird_df = weird_df.drop('MEMBER_ID', axis=1)

In [ ]:
# Count NaN values in normal_df
normal_na_count = normal_df.isna().sum()

# Count NaN values in weird_df
weird_na_count = weird_df.isna().sum()

In [ ]:
print("NaN values in normal_df:")
print(normal_na_count)

print("\nNaN values in weird_df:")
print(weird_na_count)

### NA처리

In [ ]:
normal_df.fillna(0, inplace=True)

# Fill NaN values with 0 in weird_df in-place
weird_df.fillna(0, inplace=True)

In [ ]:
normal_df.head()

### lower(), 불용어, 어근추출

In [ ]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        # Convert to lowercase
        text = text.lower()

        # Remove special characters
        text = ''.join([char for char in text if char not in string.punctuation])

    return text

def remove_stopwords(text):
    if isinstance(text, str):  # Check if the value is a string
        stop_words = set(stopwords.words('english'))
        words = text.split()
        filtered_words = [word for word in words if word.lower() not in stop_words]
        return ' '.join(filtered_words)
    return text

def apply_stemming(text):
    if isinstance(text, str):  # Check if the value is a string
        stemmer = PorterStemmer()
        words = text.split()
        stemmed_words = [stemmer.stem(word) for word in words]
        return ' '.join(stemmed_words)
    return text

# Apply text processing to specified columns
columns_to_process = ['GCATALOG_ID', 'CATALOG_ID', 'CATALOG_DESC', 'CATALOG_NM', 'KEYWORD']
for column in columns_to_process:
    normal_df[column] = normal_df[column].apply(preprocess_text)
    normal_df[column] = normal_df[column].apply(remove_stopwords)
    normal_df[column] = normal_df[column].apply(apply_stemming)
    weird_df[column] = weird_df[column].apply(preprocess_text)
    weird_df[column] = weird_df[column].apply(remove_stopwords)
    weird_df[column] = weird_df[column].apply(apply_stemming) 

In [ ]:
normal_df['KEYWORD'].value_counts()

## 상품데이터 콘캣

In [ ]:
merged_df = pd.concat([normal_df, weird_df], ignore_index=True)

# Display the merged DataFrame
merged_df['judge'].value_counts()

### REMOTEIP 컬럼 처리

In [ ]:
merged_df.columns

In [ ]:
merged_df['REMOTEIP_no_period'] = merged_df['REMOTEIP'].str.replace('.', '')
merged_df = merged_df.drop('REMOTEIP', axis=1)

In [ ]:
merged_df.head()

## CountVectorize

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vaect = CountVectorizer()
cnt_vaect.fit()